<a href="https://colab.research.google.com/github/ricotta-jpgomes/mod-multinivel-nuvem/blob/main/AWS/AWS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Catálogo de preços para VMs na AWS e suas configurações
---

**Autor**: João Paulo Gomes Ricotta


## Configuração de Ambiente

In [ ]:
!pip install ijson python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.0/135.0 kB 10.3 MB/s eta 0:00:00


In [ ]:
# Manipulação de variáveis de ambiente
from dotenv import set_key, find_dotenv, load_dotenv
import os
# Requisição à API e tratamento das respostas
import requests
import json
import ijson # Iterar sobre arquivos json gigantescos
# Manipulação de dados
import pandas as pd
import numpy as np
# Utilitários
from datetime import datetime, timezone, timedelta # Manipulação de datas
import pytz # Manipulação de fuso horário
import os # Gerenciamento do ambiente

In [ ]:
# Encontra o arquivo .env no caminho especificado
# Se não encontrar, ele pode criar um arquivo .env no diretório especificado
env_file = find_dotenv("/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Código")
if not env_file:
    env_file = '/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Código/.env'  # Cria o arquivo .env no diretório atual se não existir

In [ ]:
# obtendo a data e hora da extração dos dados
diff = timedelta(hours = -3)
tzone = timezone(diff)
extraction_date = datetime.now(tz=tzone)
extraction_date = extraction_date.astimezone(pytz.timezone("America/Sao_Paulo"))

## Coleta dos dados

### Obtendo a url para baixar o json com todas as ofertas de EC2 da Amazon

In [ ]:
# URL do arquivo de índice geral
url_indice_aws = "https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/index.json"

In [ ]:
response_indice = requests.get(url_indice_aws)
response_indice.raise_for_status() # Verifica se a requisição foi bem-sucedida

In [ ]:
print("Iniciando a coleta de dados da AWS...")

# Buscar o Índice de Ofertas da AWS
print("Buscando o arquivo de índice de ofertas da AWS...")

# URL do arquivo de índice geral
url_indice_aws = "https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/index.json"

try:
    response_indice = requests.get(url_indice_aws)
    response_indice.raise_for_status() # Verifica se a requisição foi bem-sucedida
    dados_indice = response_indice.json()

    # Extraindo a URL do arquivo de preços específico para o Amazon EC2
    url_oferta_ec2 = dados_indice['offers']['AmazonEC2']['currentVersionUrl']
    url_completa_oferta_ec2 = f"https://pricing.us-east-1.amazonaws.com{url_oferta_ec2}"

    print(f"URL da oferta do Amazon EC2 encontrada: {url_completa_oferta_ec2}")

except requests.exceptions.HTTPError as e:
    print(f"Erro HTTP ao acessar a API da AWS: {e}")
except requests.exceptions.RequestException as e:
    print(f"Erro de conexão ao acessar a API da AWS: {e}")
except KeyError:
    print("Não foi possível encontrar a oferta 'AmazonEC2' no índice da AWS.")
except json.JSONDecodeError:
    print("Erro ao decodificar a resposta JSON. O conteúdo pode não ser um JSON válido.")

Iniciando a coleta de dados da AWS...
Buscando o arquivo de índice de ofertas da AWS...
URL da oferta do Amazon EC2 encontrada: https://pricing.us-east-1.amazonaws.com/offers/v1.0/aws/AmazonEC2/current/index.json


### Baixando o json com as ofertas de EC2

In [ ]:
# Nome do arquivo temporário que será salvo no Colab
nome_arquivo_local = 'ec2_offer.json'

print("Baixando o arquivo de ofertas do EC2... (Isso pode levar alguns minutos)")

try:
    # --- Baixar o arquivo em streaming e salvar localmente ---
    print(f"Baixando o arquivo de ofertas e salvando como '{nome_arquivo_local}'...")
    print("Isso pode levar alguns minutos, mas usará pouca RAM.")

    with requests.get(url_completa_oferta_ec2, stream=True) as r:
        r.raise_for_status()
        with open(nome_arquivo_local, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

    print("Download concluído. O arquivo foi salvo no ambiente do Colab.")
except Exception as e:
    print(f"Ocorreu um erro durante o processo: {e}")
    # Se o arquivo foi criado, tenta remover em caso de erro
    if os.path.exists(nome_arquivo_local):
        os.remove(nome_arquivo_local)

Baixando o arquivo de ofertas do EC2... (Isso pode levar alguns minutos)
Baixando o arquivo de ofertas e salvando como 'ec2_offer.json'...
Isso pode levar alguns minutos, mas usará pouca RAM.
Download concluído. O arquivo foi salvo no ambiente do Colab.


In [ ]:
# Processar o arquivo local com ijson
aws_products = []
print("\nProcessando 'products' do arquivo local (configurações das instâncias)...")

with open(nome_arquivo_local, 'rb') as f:
    # Usamos ijson.kvitems para iterar sobre o dicionário 'products'
    # 'products.item' não funcionaria aqui, pois 'products' é um objeto, não uma lista
    products_parser = ijson.kvitems(f, 'products')
    for sku, attributes in products_parser:
        # Filtramos para pegar apenas as instâncias computacionais
        if attributes.get('productFamily') == 'Compute Instance':
            aws_products.append({
                'sku': sku,
                'instanceType': attributes.get('attributes', {}).get('instanceType'),
                'location': attributes.get('attributes', {}).get('location'),
                'vcpu': attributes.get('attributes', {}).get('vcpu'),
                'memory': attributes.get('attributes', {}).get('memory'),
                'operatingSystem': attributes.get('attributes', {}).get('operatingSystem'),
                'instanceFamily': attributes.get('attributes', {}).get('instanceFamily'),
                'regionCode': attributes.get('attributes', {}).get('regionCode')
            })

print(f"Processamento de produtos concluído. {len(aws_products)} SKUs de instâncias computacionais encontradas.")


Processando 'products' do arquivo local (configurações das instâncias)...
Processamento de produtos concluído. 1315358 SKUs de instâncias computacionais encontradas.


In [ ]:
for i in range(5):
  print(aws_products[i])

{'sku': 'RSH2Y67N4H4CFBQ4', 'instanceType': 'c7gd.medium', 'location': 'Asia Pacific (Malaysia)', 'vcpu': '1', 'memory': '2 GiB', 'operatingSystem': 'RHEL', 'instanceFamily': 'Compute optimized', 'regionCode': 'ap-southeast-5'}
{'sku': 'YHBHTMRVRUX8M33Y', 'instanceType': 'm7i.large', 'location': 'Asia Pacific (Malaysia)', 'vcpu': '2', 'memory': '8 GiB', 'operatingSystem': 'Red Hat Enterprise Linux with HA', 'instanceFamily': 'General purpose', 'regionCode': 'ap-southeast-5'}
{'sku': 'X6SSTWC56QWVCJCQ', 'instanceType': 'c5.24xlarge', 'location': 'Middle East (UAE)', 'vcpu': '96', 'memory': '192 GiB', 'operatingSystem': 'Windows', 'instanceFamily': 'Compute optimized', 'regionCode': 'me-central-1'}
{'sku': '8ZXHGFU4QJ2GBPJB', 'instanceType': 'c5a.large', 'location': 'AWS GovCloud (US-East)', 'vcpu': '2', 'memory': '4 GiB', 'operatingSystem': 'RHEL', 'instanceFamily': 'Compute optimized', 'regionCode': 'us-gov-east-1'}
{'sku': 'SSMSW7PJQYNKBTE4', 'instanceType': 'm6i.large', 'location': '

In [ ]:
# Preços On-Demand (Pay-As-You-Go)
aws_ondemand_terms = []
print("Extraindo preços 'On-Demand' do arquivo local...")

with open(nome_arquivo_local, 'rb') as f:
    # Apontamos o parser para o objeto 'terms.OnDemand'
    terms_parser = ijson.kvitems(f, 'terms.OnDemand')
    for sku, sku_terms in terms_parser:
        for term_code, term_details in sku_terms.items():
            price_dimensions = term_details.get('priceDimensions', {})
            for price_code, price_details in price_dimensions.items():
                # Capturamos apenas o preço por hora para instâncias em execução
                if price_details.get('unit') == 'Hrs':
                    aws_ondemand_terms.append({
                        'sku': sku,
                        'priceOnDemand': price_details.get('pricePerUnit', {}).get('USD')
                    })
print(f"-> Concluído. {len(aws_ondemand_terms)} opções de preço On-Demand encontradas.")

Extraindo preços 'On-Demand' do arquivo local...
-> Concluído. 1454084 opções de preço On-Demand encontradas.


In [ ]:
# Preços de Instâncias Reservadas
aws_reserved_terms = []
print("2. Extraindo preços 'Reserved' (1 e 3 anos) do arquivo local...")

with open(nome_arquivo_local, 'rb') as f:
    # Apontamos o parser para o objeto 'terms.Reserved'
    terms_parser = ijson.kvitems(f, 'terms.Reserved')
    for sku, sku_terms in terms_parser:
        for term_code, term_details in sku_terms.items():
            term_attributes = term_details.get('termAttributes', {})
            # Filtro crucial: Pegamos apenas a opção 'No Upfront' (sem pagamento adiantado)
            # e instâncias Standard para simplificar e garantir comparabilidade
            if (term_attributes.get('PurchaseOption') == 'No Upfront' and
                term_attributes.get('OfferingClass') == 'standard'):
                price_dimensions = term_details.get('priceDimensions', {})
                for price_code, price_details in price_dimensions.items():
                     if price_details.get('unit') == 'Hrs':
                        aws_reserved_terms.append({
                            'sku': sku,
                            'leaseContractLength': term_attributes.get('LeaseContractLength'),
                            'priceReserved': price_details.get('pricePerUnit', {}).get('USD')
                        })
print(f"-> Concluído. {len(aws_reserved_terms)} opções de preço de reserva ('No Upfront', 'Standard') encontradas.")

2. Extraindo preços 'Reserved' (1 e 3 anos) do arquivo local...
-> Concluído. 514202 opções de preço de reserva ('No Upfront', 'Standard') encontradas.


In [ ]:
# Limpeza (opcional, se quiser remover o arquivo após o uso)
# os.remove(nome_arquivo_local)
# print(f"\nArquivo temporário '{nome_arquivo_local}' removido.")

## Estruturando o dataframe

In [ ]:
# DataFrame com as configurações das instâncias
df_vms_config = pd.DataFrame(aws_products)
df_vms_config.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1315358 entries, 0 to 1315357
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   sku              1315358 non-null  object
 1   instanceType     1315357 non-null  object
 2   location         1315358 non-null  object
 3   vcpu             1315357 non-null  object
 4   memory           1315357 non-null  object
 5   operatingSystem  1315357 non-null  object
 6   instanceFamily   1315357 non-null  object
 7   regionCode       1315358 non-null  object
dtypes: object(8)
memory usage: 80.3+ MB


In [ ]:
# DataFrame com os preços On-Demand
df_vms_ondemand = pd.DataFrame(aws_ondemand_terms)
# Remove duplicatas, caso uma SKU apareça mais de uma vez
df_vms_ondemand = df_vms_ondemand.drop_duplicates(subset=['sku'])
df_vms_ondemand['priceOnDemand'] = pd.to_numeric(df_vms_ondemand['priceOnDemand'], errors='coerce') # Converte a coluna de preço para tipo numérico

df_vms_ondemand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1454084 entries, 0 to 1454083
Data columns (total 2 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   sku            1454084 non-null  object 
 1   priceOnDemand  1454084 non-null  float64
dtypes: float64(1), object(1)
memory usage: 22.2+ MB


In [ ]:
# DataFrame com os preços de instâncias reservadas
df_vms_reserved = pd.DataFrame(aws_reserved_terms)
# Remove duplicatas de SKU e prazo de contrato
df_vms_reserved = df_vms_reserved.drop_duplicates(subset=['sku', 'leaseContractLength'])
df_vms_reserved.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514202 entries, 0 to 514201
Data columns (total 3 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   sku                  514202 non-null  object
 1   leaseContractLength  514202 non-null  object
 2   priceReserved        514202 non-null  object
dtypes: object(3)
memory usage: 11.8+ MB


In [ ]:
# Converte a coluna de preço para tipo numérico, tratando possíveis erros
df_vms_reserved['priceReserved'] = pd.to_numeric(df_vms_reserved['priceReserved'], errors='coerce')

# Pivotamos a tabela para que os prazos ('1yr', '3yr') virem colunas
df_vms_reserved = df_vms_reserved.pivot_table(
    index='sku',
    columns='leaseContractLength',
    values='priceReserved'
).reset_index()

# Renomear as colunas geradas pela pivotação
df_vms_reserved.rename(columns={
    '1yr': 'priceReserved1yr',
    '3yr': 'priceReserved3yr'
}, inplace=True)

df_vms_reserved.info() # Infos do dataframe final

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259985 entries, 0 to 259984
Data columns (total 3 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   sku               259985 non-null  object 
 1   priceReserved1yr  259873 non-null  float64
 2   priceReserved3yr  254329 non-null  float64
dtypes: float64(2), object(1)
memory usage: 6.0+ MB


In [ ]:
# Merge das informações
# Começamos com o DataFrame de produtos
df_vms_aws = pd.merge(
    df_vms_config,
    df_vms_ondemand[['sku', 'priceOnDemand']], # Pegamos apenas as colunas necessárias
    on='sku',
    how='inner' # 'left' para manter todas as instâncias, mesmo que não achem um preço
)

# Agora juntamos com os preços de reserva já pivotados
df_vms_aws = pd.merge(
    df_vms_aws,
    df_vms_reserved, # Já contém 'sku', 'priceReserved1yr', 'priceReserved3yr'
    on='sku',
    how='left'
)

print("-> Processo de junção finalizado!")
# --- Visualizar o Resultado ---
print("\nAmostra do DataFrame final da AWS:")
# Exibindo colunas relevantes para a análise de preço
display(df_vms_aws[[
    'instanceType',
    'location',
    'vcpu',
    'memory',
    'priceOnDemand',
    'priceReserved1yr',
    'priceReserved3yr'
]].head())

print("\nInformações sobre o DataFrame final:")
df_vms_aws.info()

-> Processo de junção finalizado!

Amostra do DataFrame final da AWS:


,instanceType,location,vcpu,memory,priceOnDemand,priceReserved1yr,priceReserved3yr
0,c7gd.medium,Asia Pacific (Malaysia),1,2 GiB,0.0000,NaN,NaN
1,m7i.large,Asia Pacific (Malaysia),2,8 GiB,0.0000,NaN,NaN
2,c5.24xlarge,Middle East (UAE),96,192 GiB,9.4850,7.609,6.606
3,c5a.large,AWS GovCloud (US-East),2,4 GiB,0.0000,NaN,NaN
4,m6i.large,AWS GovCloud (US-West),2,8 GiB,0.2174,NaN,NaN



Informações sobre o DataFrame final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1314550 entries, 0 to 1314549
Data columns (total 11 columns):
 #   Column            Non-Null Count    Dtype  
---  ------            --------------    -----  
 0   sku               1314550 non-null  object 
 1   instanceType      1314549 non-null  object 
 2   location          1314550 non-null  object 
 3   vcpu              1314549 non-null  object 
 4   memory            1314549 non-null  object 
 5   operatingSystem   1314549 non-null  object 
 6   instanceFamily    1314549 non-null  object 
 7   regionCode        1314550 non-null  object 
 8   priceOnDemand     1314550 non-null  float64
 9   priceReserved1yr  235296 non-null   float64
 10  priceReserved3yr  229752 non-null   float64
dtypes: float64(3), object(8)
memory usage: 110.3+ MB


In [ ]:
df_vms_aws = df_vms_aws.loc[
    (df_vms_aws['priceOnDemand'].notna()) &
    (df_vms_aws['priceOnDemand'] > 0)
]

print(f"-> Quantidade de registros após o filtro: {len(df_vms_aws)}")

-> Quantidade de registros após o filtro: 795174


## Tratamento prévio e persistência dos dados

In [ ]:
# Filtro para VMs de propóstio geral
aws_proposito_geral = ('m8g', 'm7g', 'm7i', 'm7i-flex', 'm7a', 'mac', 'm6g', 'm6i', 'm6in', 'm6a', 'm5', 'm5n', 'm5zn', 'm5a', 'm4', 't4g', 't3', 't3a', 't2')

df_vms_aws = df_vms_aws.loc[(df_vms_aws['instanceFamily'] == 'General purpose') &
                (df_vms_aws['instanceType'].str.startswith(aws_proposito_geral))
              ]

print(f"-> Quantidade de registros após o filtro: {len(df_vms_aws)}")
df_vms_aws.info()
display(df_vms_aws.head())

-> Quantidade de registros após o filtro: 225177
<class 'pandas.core.frame.DataFrame'>
Index: 225177 entries, 4 to 1314546
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   sku               225177 non-null  object 
 1   instanceType      225177 non-null  object 
 2   location          225177 non-null  object 
 3   vcpu              225177 non-null  object 
 4   memory            225177 non-null  object 
 5   operatingSystem   225177 non-null  object 
 6   instanceFamily    225177 non-null  object 
 7   regionCode        225177 non-null  object 
 8   priceOnDemand     225177 non-null  float64
 9   priceReserved1yr  66752 non-null   float64
 10  priceReserved3yr  66752 non-null   float64
dtypes: float64(3), object(8)
memory usage: 20.6+ MB


,sku,instanceType,location,vcpu,memory,operatingSystem,instanceFamily,regionCode,priceOnDemand,priceReserved1yr,priceReserved3yr
4,SSMSW7PJQYNKBTE4,m6i.large,AWS GovCloud (US-West),2,8 GiB,RHEL,General purpose,us-gov-west-1,0.21740,NaN,NaN
11,6P49AUTUVZ7E2G82,m7i-flex.12xlarge,AWS GovCloud (US-West),48,192 GiB,Ubuntu Pro,General purpose,us-gov-west-1,2.98080,NaN,NaN
13,EJ56CUQYBC3GEZMZ,m7g.large,Asia Pacific (Seoul),2,8 GiB,Ubuntu Pro,General purpose,ap-northeast-2,0.11000,NaN,NaN
27,3FB266G79G8SPDUP,m6idn.12xlarge,Asia Pacific (Singapore),48,192 GiB,RHEL,General purpose,ap-southeast-1,5.67670,NaN,NaN
39,Z4Y9ZT5DGBQCTUP6,m6id.large,US East (Ohio),2,8 GiB,Red Hat Enterprise Linux with HA,General purpose,us-east-2,0.67432,NaN,NaN


In [ ]:
df_vms_aws.drop(columns=['sku', 'operatingSystem', 'instanceFamily'], inplace=True)

In [ ]:
df_vms_aws['memory'] = pd.to_numeric(df_vms_aws['memory'].str.replace(' GiB', ''))
df_vms_aws['memory'] = df_vms_aws['memory'] * 1024
df_vms_aws['memory'] = df_vms_aws['memory'].astype('int64')

print("-> Processo de conversão concluído com sucesso:")
display(df_vms_aws['memory'].head())

-> Processo de conversão concluído com sucesso:


,memory
4,8192
11,196608
13,8192
27,196608
39,8192


In [ ]:
df_vms_aws[['priceOnDemand', 'priceReserved1yr', 'priceReserved3yr']] = round(df_vms_aws[['priceOnDemand', 'priceReserved1yr', 'priceReserved3yr']], 3)

In [ ]:
df_vms_aws.rename(
    columns = {
      'instanceType': 'machineType',
      'location': 'region',
      'vcpu': 'numberOfCores',
      'memory': 'memoryInMiB',
  }, inplace = True
)

df_vms_aws.head()

,machineType,region,numberOfCores,memoryInMiB,regionCode,priceOnDemand,priceReserved1yr,priceReserved3yr
4,m6i.large,AWS GovCloud (US-West),2,8192,us-gov-west-1,0.217,NaN,NaN
11,m7i-flex.12xlarge,AWS GovCloud (US-West),48,196608,us-gov-west-1,2.981,NaN,NaN
13,m7g.large,Asia Pacific (Seoul),2,8192,ap-northeast-2,0.110,NaN,NaN
27,m6idn.12xlarge,Asia Pacific (Singapore),48,196608,ap-southeast-1,5.677,NaN,NaN
39,m6id.large,US East (Ohio),2,8192,us-east-2,0.674,NaN,NaN


In [ ]:
df_vms_aws['extractionDate'] = extraction_date
df_vms_aws['provider'] = 'AWS'
df_vms_aws['currencyCode'] = 'USD'

In [ ]:
ordered_columns = ['extractionDate', 'provider', 'machineType', 'regionCode', 'region', 'numberOfCores', 'memoryInMiB', 'currencyCode', 'priceOnDemand', 'priceReserved1yr', 'priceReserved3yr']
df_vms_aws = df_vms_aws[ordered_columns]
df_vms_aws.info()

<class 'pandas.core.frame.DataFrame'>
Index: 225177 entries, 4 to 1314546
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype                            
---  ------            --------------   -----                            
 0   extractionDate    225177 non-null  datetime64[us, America/Sao_Paulo]
 1   provider          225177 non-null  object                           
 2   machineType       225177 non-null  object                           
 3   regionCode        225177 non-null  object                           
 4   region            225177 non-null  object                           
 5   numberOfCores     225177 non-null  object                           
 6   memoryInMiB       225177 non-null  int64                            
 7   currencyCode      225177 non-null  object                           
 8   priceOnDemand     225177 non-null  float64                          
 9   priceReserved1yr  66752 non-null   float64                          
 10  

In [ ]:
file_name = f'vms_aws_{extraction_date.strftime("%d-%m-%Y")}.csv'
set_key(env_file, "AWS_FILE", file_name)

df_vms_aws.to_csv(f"/content/drive/MyDrive/MBA Data Science e Analytics/TCC/Dados/{file_name}", index=False)